In [ ]:
#| default_exp gnncoal

In [ ]:
#| export

import torch
import torch.nn.functional as F
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv

from math import ceil
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import dense_diff_pool
from torch_geometric.loader import DenseDataLoader
from torch_geometric.loader import DataLoader
from torch_geometric.nn.dense.dense_gcn_conv import DenseGCNConv

from torch_geometric.utils.to_dense_adj import to_dense_adj
from torch_geometric.utils.to_dense_batch import to_dense_batch
from x_transformers import Encoder


In [ ]:
#| export 

class GNN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, normalize=True, lin=True, track_running_stats=True):
        super().__init__()
        
        self.conv1 = DenseGCNConv(in_channels, hidden_channels, normalize)
        self.bn1 = nn.BatchNorm1d(hidden_channels, track_running_stats=track_running_stats)
        
        self.conv2 = DenseGCNConv(hidden_channels, hidden_channels, normalize)
        self.bn2 = nn.BatchNorm1d(hidden_channels, track_running_stats=track_running_stats)
        
        self.conv3 = DenseGCNConv(hidden_channels, out_channels, normalize)
        self.bn3 = nn.BatchNorm1d(out_channels, track_running_stats=track_running_stats)
        
        if lin is True:
            self.lin = nn.Linear(2*hidden_channels + out_channels, out_channels)
        else:
            self.lin = None
            
    def bn(self, i, x):
        batch_size, num_nodes, num_channels = x.size()
        x = x.view(-1, num_channels)
        x = getattr(self, f'bn{i}')(x)
        x = x.view(batch_size, num_nodes, num_channels)
        return x

    def forward(self, x, adj, mask=None):
        batch_size, num_nodes, in_channels = x.size()

        x0 = x
        x1 = self.bn(1, F.relu(self.conv1(x0, adj, mask)))
        x2 = self.bn(2, F.relu(self.conv2(x1, adj, mask)))
        x3 = self.bn(3, F.relu(self.conv3(x2, adj, mask)))
        
        x = torch.cat([x1, x2, x3], dim=-1)

        if self.lin is not None:
            x = F.relu(self.lin(x))

        return x

In [ ]:
#| export 
#torch.Tensor(0)



            
class DiffPoolNet(torch.nn.Module):
    def __init__(self, max_nodes, num_features, num_hidden=64, out_channels=60, track_running_stats=True):
        super().__init__()

        num_nodes = ceil(0.3 * max_nodes)
        self.gnn1_pool = GNN(num_features, num_hidden, num_nodes, track_running_stats=track_running_stats)
        self.gnn1_embed = GNN(num_features, num_hidden, num_hidden, lin=False, track_running_stats=track_running_stats)

        num_nodes = ceil(0.3 * num_nodes)
        self.gnn2_pool = GNN(3 * num_hidden, num_hidden, num_nodes, track_running_stats=track_running_stats)
        self.gnn2_embed = GNN(3 * num_hidden, num_hidden, num_hidden, lin=False, track_running_stats=track_running_stats)
        
        num_nodes = ceil(0.3 * num_nodes)
        self.gnn3_pool = GNN(3 * num_hidden, num_hidden, num_nodes, track_running_stats=track_running_stats)
        self.gnn3_embed = GNN(3 * num_hidden, num_hidden, num_hidden, lin=False, track_running_stats=track_running_stats)
        

        self.gnn4_embed = GNN(3 * num_hidden, num_hidden, num_hidden, lin=False, track_running_stats=track_running_stats)

        self.lin1 = nn.Linear(3 * num_hidden, num_hidden)        
        self.lin2 = nn.Linear(num_hidden, out_channels)
        

    def forward(self, x, adj):
        
        s = self.gnn1_pool(x, adj)
        x = self.gnn1_embed(x, adj)
        x, adj, l1, e1 = dense_diff_pool(x, adj, s)
         
        s = self.gnn2_pool(x, adj)
        x = self.gnn2_embed(x, adj)
        x, adj, l2, e2 = dense_diff_pool(x, adj, s)

        s = self.gnn3_pool(x, adj)
        x = self.gnn3_embed(x, adj)
        x, adj, l3, e3 = dense_diff_pool(x, adj, s)
        
        x = self.gnn4_embed(x, adj)

        x = x.mean(dim=1)        
        x = F.relu(self.lin1(x))
        x = self.lin2(x)
                                            
        return x

In [ ]:
#| export

class MiniDenseNet(torch.nn.Module):
    def __init__(self, in_features, hidden_features, out_features):
        super(MiniDenseNet, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(in_features, hidden_features),
            nn.ReLU(),
            nn.Linear(hidden_features, hidden_features),
            nn.ReLU(),
            nn.Linear(hidden_features, out_features),
        )

    def forward(self, x):
        return self.net(x)

NameError: name 'torch' is not defined

In [ ]:
#| export 
#torch.Tensor(0)



            
class DiffPoolNetV2(torch.nn.Module):
    def __init__(self, max_nodes, num_features, num_hidden=64, out_channels=60, num_trees=500, enc_depth=12, enc_heads=8, track_running_stats=True):
        super().__init__()

        num_nodes = ceil(0.3 * max_nodes)
        self.gnn1_pool = GNN(num_features, num_hidden, num_nodes, track_running_stats=track_running_stats)
        self.gnn1_embed = GNN(num_features, num_hidden, num_hidden, lin=False, track_running_stats=track_running_stats)

        num_nodes = ceil(0.3 * num_nodes)
        self.gnn2_pool = GNN(3 * num_hidden, num_hidden, num_nodes, track_running_stats=track_running_stats)
        self.gnn2_embed = GNN(3 * num_hidden, num_hidden, num_hidden, lin=False, track_running_stats=track_running_stats)
        
        num_nodes = ceil(0.3 * num_nodes)
        self.gnn3_pool = GNN(3 * num_hidden, num_hidden, num_nodes, track_running_stats=track_running_stats)
        self.gnn3_embed = GNN(3 * num_hidden, num_hidden, num_hidden, lin=False, track_running_stats=track_running_stats)
        

        self.gnn4_embed = GNN(3 * num_hidden, num_hidden, num_hidden, lin=False, track_running_stats=track_running_stats)

        enc_dim = num_hidden * 3
        self.encoder = Encoder(dim=enc_dim, depth=enc_depth, heads=enc_heads,
                               ff_glu=True, residual_attn=True, flash=True)

        self.positional_encodings = nn.Parameter(torch.randn(num_trees, enc_dim))
        self.demography_head = MiniDenseNet(enc_dim, enc_dim//2, out_channels)

    def forward(self, x, adj, batch_size, num_trees=500):
        
        s = self.gnn1_pool(x, adj)
        x = self.gnn1_embed(x, adj)
        x, adj, l1, e1 = dense_diff_pool(x, adj, s)
         
        s = self.gnn2_pool(x, adj)
        x = self.gnn2_embed(x, adj)
        x, adj, l2, e2 = dense_diff_pool(x, adj, s)

        s = self.gnn3_pool(x, adj)
        x = self.gnn3_embed(x, adj)
        x, adj, l3, e3 = dense_diff_pool(x, adj, s)
        
        x = self.gnn4_embed(x, adj)

        x = x.mean(dim=1)

        x = x.view(batch_size, num_trees, x.shape[-1])
        x = x + self.positional_encodings.unsqueeze(0)
        x = self.encoder(x)  
        x = x.sum(dim=1)
        x = self.demography_head(x)
                                            
        return x

In [ ]:
#| export 
#torch.Tensor(0)

            
class DiffPoolNetV3(torch.nn.Module):
    def __init__(self, max_nodes, num_features, num_hidden=64, out_channels=60, num_trees=500, enc_depth=12, enc_heads=8, track_running_stats=True):
        super().__init__()

        num_nodes = ceil(0.7 * max_nodes)
        self.gnn1_pool = GNN(num_features, num_hidden, num_nodes, track_running_stats=track_running_stats)
        self.gnn1_embed = GNN(num_features, num_hidden, num_hidden, lin=False, track_running_stats=track_running_stats)

        num_nodes = ceil(0.6 * num_nodes)
        self.gnn2_pool = GNN(3 * num_hidden, num_hidden, num_nodes, track_running_stats=track_running_stats)
        self.gnn2_embed = GNN(3 * num_hidden, num_hidden, num_hidden, lin=False, track_running_stats=track_running_stats)
        
        num_nodes = ceil(0.5 * num_nodes)
        self.gnn3_pool = GNN(3 * num_hidden, num_hidden, num_nodes, track_running_stats=track_running_stats)
        self.gnn3_embed = GNN(3 * num_hidden, num_hidden, num_hidden, lin=False, track_running_stats=track_running_stats)
        

        self.gnn4_embed = GNN(3 * num_hidden, num_hidden, num_hidden, lin=False, track_running_stats=track_running_stats)

        enc_dim = num_hidden * 3
        self.encoder = Encoder(dim=enc_dim, depth=enc_depth, heads=enc_heads,
                               ff_glu=True, residual_attn=True, flash=True)

        self.positional_encodings = nn.Parameter(torch.randn(num_trees, enc_dim))
        self.demography_head = MiniDenseNet(enc_dim, enc_dim//2, out_channels)

    def forward(self, x, adj, batch_size, num_trees=500):
        
        s = self.gnn1_pool(x, adj)
        x = self.gnn1_embed(x, adj)
        x, adj, l1, e1 = dense_diff_pool(x, adj, s)
         
        s = self.gnn2_pool(x, adj)
        x = self.gnn2_embed(x, adj)
        x, adj, l2, e2 = dense_diff_pool(x, adj, s)

        s = self.gnn3_pool(x, adj)
        x = self.gnn3_embed(x, adj)
        x, adj, l3, e3 = dense_diff_pool(x, adj, s)
        
        x = self.gnn4_embed(x, adj)
        x = x.sum(dim=1)
        x = x.view(batch_size, num_trees, x.shape[-1])
        x = x + self.positional_encodings.unsqueeze(0)
        x = self.encoder(x)  
        x = x.sum(dim=1)
        x = self.demography_head(x)
                                            
        return x